In [80]:
import pandas as pd

In [81]:
data = pd.read_csv("/Users/dariashavronskaia/Desktop/data science/data/weatherAUS.csv")

In [82]:
data.isna().sum().sum()

343248

In [83]:
data.shape

(145460, 23)

In [84]:
mask = data.isna().sum()/len(data) > 0.4

In [85]:
mask

Date             False
Location         False
MinTemp          False
MaxTemp          False
Rainfall         False
Evaporation       True
Sunshine          True
WindGustDir      False
WindGustSpeed    False
WindDir9am       False
WindDir3pm       False
WindSpeed9am     False
WindSpeed3pm     False
Humidity9am      False
Humidity3pm      False
Pressure9am      False
Pressure3pm      False
Cloud9am         False
Cloud3pm          True
Temp9am          False
Temp3pm          False
RainToday        False
RainTomorrow     False
dtype: bool

In [86]:
data_clean = data.drop(['Evaporation', 'Sunshine', 'Cloud3pm'], axis = 1)

In [87]:
data_clean.shape

(145460, 20)

In [88]:
data_clean['RainToday'] = data_clean['RainToday'].map({'No': 0, 'Yes': 1})
data_clean['RainTomorrow'] = data_clean['RainTomorrow'].map({'No': 0, 'Yes': 1})

In [89]:
data_clean['RainToday'].sum()/len(data_clean['RainToday'])

0.2191667812457033

In [90]:
data_clean['Date'] = pd.to_datetime(data_clean['Date'])

In [91]:
data_clean['Month'] = data_clean['Date'].dt.month

In [92]:
data_clean['Month']

0         12
1         12
2         12
3         12
4         12
          ..
145455     6
145456     6
145457     6
145458     6
145459     6
Name: Month, Length: 145460, dtype: int32

In [93]:
data_clean.drop('Date', axis = 1, inplace = True)

In [94]:
data_season = data_clean.groupby('Month').mean(numeric_only=True)

In [95]:
data_season[['RainToday']]


,RainToday
Month,
1,0.189484
2,0.206746
3,0.217135
4,0.216845
5,0.222163
6,0.263638
7,0.270736
8,0.253167
9,0.229135


In [96]:
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
data_dummies = pd.get_dummies(data_clean, columns=categoricals)
data_dummies.shape

(145460, 124)

In [98]:
data_dummies.dropna(inplace=True)
X = data_dummies.drop('RainTomorrow', axis = 1)
y = data_dummies['RainTomorrow']  

In [99]:
from sklearn.model_selection import *
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31)

In [100]:
y_test.mean()

0.22770253002811142

In [101]:
import numpy as np
np.random.seed(31)

In [102]:
def gbs(data, n):     
    inds = np.random.randint(0, len(data), (n, len(data)))
    numbers = data[inds]
    return numbers
target = X_train['MinTemp'].values 
np.random.seed(31) 
mean_values = [np.mean(x) for x in gbs(target, 1000)]
np.std(mean_values) 

0.02879072820657669

In [111]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [107]:
clf = LogisticRegression()

In [108]:
clf.fit(X_train, y_train)

/Users/dariashavronskaia/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [109]:
pred_train = clf.predict(X_train)

In [110]:
pred_test = clf.predict(X_test)

In [112]:
roc_auc_score(y_test, pred_test)

0.7264688889532628

In [113]:
params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]}

In [114]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state=42)
grid_search_cv = GridSearchCV(clf, param_grid=params, cv=3, verbose=3)
grid_search_cv.fit(X_train, y_train)


Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.817 total time=   0.1s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.820 total time=   0.1s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=2;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.817 total time=   0.1s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.820 total time=   0.1s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=3;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.817 total time=   0.1s
[CV 2/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.820 total time=   0.1s
[CV 3/3] END max_depth=5, max_leaf_nodes=2, min_samples_split=4;, score=0.825 total time=   0.1s
[CV 1/3] END max_depth=5, max_leaf_nodes=3, min_samples_split=2;,

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [5, 7, 9, 11],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_split': [2, 3, 4]},
             verbose=3)

In [115]:
grid_search_cv.best_params_

{'max_depth': 5, 'max_leaf_nodes': 9, 'min_samples_split': 2}

In [116]:
clf = tree.DecisionTreeClassifier(max_depth=5, max_leaf_nodes=9, min_samples_split=2, random_state=42)
clf.fit(X_train, y_train)
pred_test = clf.predict(X_test)
pred_train = clf.predict(X_train)
roc_auc_score(y_test, pred_test)

0.7033229072349596

In [117]:
from sklearn.ensemble import RandomForestClassifier

In [120]:
rfc = RandomForestClassifier(max_depth=100, random_state=31)
rfc.fit(X_train, y_train)
pred_test = rfc.predict(X_test)
pred_train = rfc.predict(X_train)

In [121]:
roc_auc_score(y_test, pred_test)

0.7326602978581066

In [123]:
rfc = RandomForestClassifier(random_state = 31)
params = {'max_features': [ 4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]}
grid_search_cv = GridSearchCV(rfc, param_grid=params, verbose=3, cv=3)
grid_search_cv.fit(X_train, y_train)
grid_search_cv.best_params_

Fitting 3 folds for each of 60 candidates, totalling 180 fits
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.776 total time=   0.7s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   0.7s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=3;, score=0.775 total time=   0.6s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.776 total time=   0.6s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   0.6s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=5;, score=0.775 total time=   0.6s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.776 total time=   0.6s
[CV 2/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   0.6s
[CV 3/3] END max_depth=5, max_features=4, min_samples_leaf=7;, score=0.775 total time=   0.6s
[CV 1/3] END max_depth=5, max_features=4, min_samples_leaf=9;, score=0.776 total time=   0.6

{'max_depth': 15, 'max_features': 7, 'min_samples_leaf': 3}

In [124]:
rfc = RandomForestClassifier(max_depth=15, max_features=7, min_samples_leaf=3, random_state = 31)
rfc.fit(X_train, y_train)
pred_test = rfc.predict(X_test)
pred_train = rfc.predict(X_train)
roc_auc_score(y_test, pred_test)

0.699965714700254

In [126]:
feature_names = [x for x in data_dummies if x != 'RainTomorrow']
pd.DataFrame({'feat': feature_names,
              'coef': rfc.feature_importances_}).sort_values(by='coef', ascending=False)

,feat,coef
7,Humidity3pm,0.247119
2,Rainfall,0.080664
6,Humidity9am,0.074188
9,Pressure3pm,0.065947
10,Cloud9am,0.065545
...,...,...
40,Location_GoldCoast,0.000000
69,Location_Walpole,0.000000
43,Location_Launceston,0.000000
72,Location_Witchcliffe,0.000000
